<a href="https://colab.research.google.com/github/kty0307/Blog/blob/main/SP_LOAN_INTRATE_DOWNLOAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **연월입력**

In [36]:
year = 2024     ## ex) 2024
month = 2       ## ex) 1

# 0. 환경설정

In [23]:
import requests
import json
import pandas as pd
import urllib.parse
import urllib3
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import io
from google.colab import files

In [24]:
class TLSAdapter(requests.adapters.HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("AES128-SHA256")
        kwargs["ssl_context"] = ctx
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)

In [25]:
class NewTLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("AES128-SHA256")  # 원하는 Cipher Suite 설정
        kwargs["ssl_context"] = ctx
        return super().init_poolmanager(*args, **kwargs)

In [26]:
def subtract_month(df, col_name, new_col_name):
    df[new_col_name] = pd.to_datetime(df[col_name], format='%Y%m') - pd.DateOffset(months=1)
    df[new_col_name] = df[new_col_name].dt.strftime('%Y%m').astype(int)
    return df

# 1. 저축은행

## 1.1. 신용점수별 금리

In [27]:
def savings_cs(year, month):
  url = "https://www.fsb.or.kr/ratloanconf_0200.jct"
  payload = {
      "_JSON_": json.dumps({
          "SORT_COLUMN": "",
          "SORT": "",
          "PRE_MONTH_MONEY": "",
          "SUBMIT_MONTH": f"{year:04d}{month:02d}"
      })
  }

  response = requests.post(url, data=payload)

  data = response.text

  json_data = json.loads(data)

  # 데이터 추출 및 DataFrame 생성
  df = pd.DataFrame(json_data['REC'])
  df = df[['BANK_NAME', 'SUBMIT_MONTH', 'A_RATE1_3', 'A_RATE1', 'A_RATE2',
            'A_RATE3', 'A_RATE_AVE']]

  df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

  df['업권'] = '저축은행업권'

  df = subtract_month(df, '공시연월', '연월')

  df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

  return df

## 1.2. 금리대별 취급비중

In [18]:
def savings_int(year, month):
    url = "https://www.fsb.or.kr/ratloanconf_0300.jct"

    payload = {
        "SORT": "",
        "SUBMIT_MONTH": f"{year}{month:02}"
    }

    response = requests.post(url, data={"_JSON_": json.dumps(payload)})
    response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    df = pd.DataFrame(data["REC"])  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    desired_columns = ["BANK_NAME", "SUBMIT_MONTH", "HANDING_WEIGHT_10", "HANDING_WEIGHT_12", "HANDING_WEIGHT_14"
                        ,"HANDING_WEIGHT_16", "HANDING_WEIGHT_18", "HANDING_WEIGHT_20"]
    df = df[desired_columns] #원하는 열만 남기기

    df.columns = ['사명','제출연월','10%이하','12%이하','14%이하','16%이하','18%이하','20%이하']

    df['업권'] = '저축은행업권'

    return df

# 2. 신용카드

## 2.1. 카드론_신용점수별 금리

In [28]:
def card_loan_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditcard/creditcardDisclosureDetail25Ajax"

    # cgc_seq(월)을 찾기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": 1458,
        "cgcMode": 25,
        "cgcYyyy": year,
        "mcSeq": []
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['cgcquarter'] == month:
            cgc_seq = item['cgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": cgc_seq,
        "cgcMode": 25,
        "cgcYyyy": year,
        "mcSeq": [31, 96, 1, 106, 14, 13, 12, 98, 502, 108, 619, 11, 97, 105, 103, 22]
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "cgcSeq", "cgCardPoint1", "cgCardPoint2", "cgCardPoint3"
                ,"cgCardPoint4", "cgCardPointAvg"]] #원하는 열만 남기기

    df["cgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

    df['업권'] = '신용카드업권(카드론)'

    df = subtract_month(df, '공시연월', '연월')

    df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

    return df

## 2.2. 현금서비스_신용점수별 금리

In [29]:
def card_cash_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditcard/creditcardDisclosureDetail20Ajax"

    # cgc_seq(월)을 찾기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": 1460,
        "cgcMode": 20,
        "cgcYyyy": year,
        "mcSeq": []
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['cgcquarter'] == month:
            cgc_seq = item['cgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": cgc_seq,
        "cgcMode": 20,
        "cgcYyyy": year,
        "mcSeq": [31, 96, 1, 106, 14, 13, 12, 98, 502, 108, 619, 11, 97, 105, 103, 22]
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "cgcSeq", "cgMoneyPoint1", "cgMoneyPoint2", "cgMoneyPoint3"
                ,"cgMoneyPoint4", "cgMoneyPointAvg"]] #원하는 열만 남기기

    df["cgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

    df['업권'] = '신용카드업권(현금서비스)'

    df = subtract_month(df, '공시연월', '연월')

    df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

    return df

# 3. 캐피탈

## 3.1. 신용점수별 금리

In [30]:
def capital_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditloan/creditloanDisclosureDetail11/ajax"

    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
          "clgcMode": 11,
          "cardItem": "134,39,40,623,130,41,25,156,6,55,32,58,52,61,57,64",
          "clgcSeq": 521,
          "clgcYyyy": 2024
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['clgcquarter'] == month:
            clgc_seq = item['clgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
          "clgcMode": 11,
          "cardItem": "134,39,40,623,130,41,25,156,6,55,32,58,52,61,57,64",
          "clgcSeq": clgc_seq,
          "clgcYyyy": year
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "clgcSeq", "clgPoint1", "clgPoint2", "clgPoint3"
                ,"clgPoint4", "clgPointAvg"]] #원하는 열만 남기기

    df["clgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

    df['업권'] = '캐피탈업권'

    df = subtract_month(df, '공시연월', '연월')

    df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

    return df


# 4. 은행

## 4.1. 신용점수별 금리

In [31]:
def bank_cs(year, month):
    url = "https://portal.kfb.or.kr/compare/loan_household_new_search_result_new.php"

    payload = {
        "year": year,
        "month": f"{month:02}",
        "opt_1": 3,
        "detail": 0,
        "str": "KDB%BB%EA%BE%F7%C0%BA%C7%E0|NH%B3%F3%C7%F9%C0%BA%C7%E0|%BD%C5%C7%D1%C0%BA%C7%E0|%BF%EC%B8%AE%C0%BA%C7%E0|SC%C1%A6%C0%CF%C0%BA%C7%E0|%C7%CF%B3%AA%C0%BA%C7%E0|IBK%B1%E2%BE%F7%C0%BA%C7%E0|KB%B1%B9%B9%CE%C0%BA%C7%E0|%C7%D1%B1%B9%BE%BE%C6%BC%C0%BA%C7%E0|Sh%BC%F6%C7%F9%C0%BA%C7%E0|iM%B9%F0%C5%A9%28%B1%B8+%B4%EB%B1%B8%C0%BA%C7%E0%29|BNK%BA%CE%BB%EA%C0%BA%C7%E0|%B1%A4%C1%D6%C0%BA%C7%E0|%C1%A6%C1%D6%C0%BA%C7%E0|%C0%FC%BA%CF%C0%BA%C7%E0|BNK%B0%E6%B3%B2%C0%BA%C7%E0|%C4%C9%C0%CC%B9%F0%C5%A9|%C4%AB%C4%AB%BF%C0%B9%F0%C5%A9|%C5%E4%BD%BA%B9%F0%C5%A9",
        "select_new_balance": 1
    }

    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Host": "portal.kfb.or.kr",
        "Origin": "https://portal.kfb.or.kr",
        "Referer": "https://portal.kfb.or.kr/compare/loan_household_new.php",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest",
    }
    with requests.session() as s:
        s.mount("https://", TLSAdapter())
        try:
            response = s.post(url, data=payload, headers=headers)
            response.raise_for_status()
            response.encoding = 'euc-kr'

        except requests.exceptions.RequestException as e:
            print(f"요청 실패: {e}")
            return None

    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find('table')
    data = []
    rows = table.find_all('tr')[2:]
    for row in rows:
        cols = [col.text.strip() for col in row.find_all('td')]
        cols[1] = cols[1] + cols[2]
        # cols.pop(2)
        data.append(cols)

    df = pd.DataFrame(data)

    df = df[[0,2,3,4,5,6,7,8,9,11]]

    for col in [2,3,4,5,6,7,8,9,11]:
      df[col] = pd.to_numeric(df[col], errors='coerce')

    df['900점대'] = df[[2, 3]].mean(axis=1)
    df['800점대'] = df[[4, 5]].mean(axis=1)
    df['700점대'] = df[[6, 7]].mean(axis=1)
    df['600점대'] = df[[8, 9]].mean(axis=1)

    df['업권'] = df[0].apply(lambda x:
        '은행업권_지방' if any(city in x for city in ['대구', '부산', '광주', '제주', '전북', '경남']) else
        '은행업권_인터넷' if any(bank in x for bank in ['케이', '카카오', '토스']) else
        '은행업권_특수' if any(spc in x for spc in ['수협', '기업']) else
        '은행업권_시중'
    )
    df['공시연월'] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    df = subtract_month(df, '공시연월', '연월')

    df.rename(columns={0: '사명', 11:'평균금리'}, inplace=True)

    df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

    df.dropna(subset=['평균금리'], inplace=True)

    return df

# 5. 데이터 처리 및 다운로드

In [37]:
file_names = f'SP_INTRATE_{year}{month:02d}.csv'

In [38]:
dfs = []

try:
  dfs.append(savings_cs(year, month))
except Exception:
  pass
try:
  dfs.append(card_loan_cs(year, month))
except Exception:
  pass
try:
  dfs.append(card_cash_cs(year, month))
except Exception:
  pass
try:
  dfs.append(capital_cs(year, month))
except Exception:
  pass
try:
  dfs.append(bank_cs(year, month))
except Exception:
  pass

result = pd.concat(dfs, ignore_index=True)
result.to_csv(file_names, index=False, encoding='euc-kr')
files.download(file_names)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 6. DB생성_업데이트에는 사용하지 않음

In [ ]:
# start_year = 2020
# start_month = 2
# end_year = 2025
# end_month = 2

# dfs = []
# for year in range(start_year, end_year + 1):
#     start = start_month if year == start_year else 1
#     end = end_month + 1 if year == end_year else 13
#     for month in range(start, end):
#         try:
#           dfs.append(savings_cs(year, month))
#         except Exception:
#           pass
#         try:
#           dfs.append(card_loan_cs(year, month))
#         except Exception:
#           pass
#         try:
#           dfs.append(card_cash_cs(year, month))
#         except Exception:
#           pass
#         try:
#           dfs.append(capital_cs(year, month))
#         except Exception:
#           pass
#         try:
#           dfs.append(bank_cs(year, month))
#         except Exception:
#           pass

# result = pd.concat(dfs, ignore_index=True)
# result.to_csv(file_names, index=False, encoding='euc-kr')
# files.download(file_names)